# item.csv 파일 전처리
----
- 불필요한 칼럼 제거
- 결측치 처리
- 문자형/리스트형 칼럼에 대한 인코딩
- Scaler 사용한 수치형 칼럼 단위 통일

In [ ]:
# 데이터 불러오기
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/data/preprocessing/item.csv", low_memory=False)

In [ ]:
# 전체 칼럼에 대한 정보 확인 -> 113 columns
df.info(max_cols=200)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063724 entries, 0 to 1063723
Data columns (total 113 columns):
 #    Column                             Non-Null Count    Dtype  
---   ------                             --------------    -----  
 0    nickname                           1063724 non-null  object 
 1    world                              1063724 non-null  object 
 2    level                              1063724 non-null  int64  
 3    job                                1063724 non-null  object 
 4    subclass                           1063724 non-null  object 
 5    preset_number                      1063724 non-null  int64  
 6    equipment_part                     1063724 non-null  object 
 7    equipment_slot                     1063724 non-null  object 
 8    item_name                          1063724 non-null  object 
 9    str_total                          1063724 non-null  int64  
 10   dex_total                          1063724 non-null  int64  
 11   int_total

## 불필요한 칼럼 제거

In [ ]:
# 불필요한 칼럼 드랍: 장비의 기본 능력치 정보
cols_to_drop = [
    "str_base", "dex_base", "int_base", "luk_base",
    "max_hp_base", "max_mp_base",
    "attack_power_base", "magic_power_base", "armor_base",
    "speed_base", "jump_base",
    "boss_damage_base", "ignore_monster_armor_base",
    "all_stat_base", "max_hp_rate_base", "max_mp_rate_base",
    "base_equipment_level_base"
]

df.drop(columns=cols_to_drop, inplace=True)

In [ ]:
df.info() # 96 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063724 entries, 0 to 1063723
Data columns (total 96 columns):
 #   Column                             Non-Null Count    Dtype  
---  ------                             --------------    -----  
 0   nickname                           1063724 non-null  object 
 1   world                              1063724 non-null  object 
 2   level                              1063724 non-null  int64  
 3   job                                1063724 non-null  object 
 4   subclass                           1063724 non-null  object 
 5   preset_number                      1063724 non-null  int64  
 6   equipment_part                     1063724 non-null  object 
 7   equipment_slot                     1063724 non-null  object 
 8   item_name                          1063724 non-null  object 
 9   str_total                          1063724 non-null  int64  
 10  dex_total                          1063724 non-null  int64  
 11  int_total               

### 장비 최종 옵션
----
전략: 주스탯에 맞지 않는 능력치 칼럼 제거 후 하나의 열로 통일

In [ ]:
# 직업별 주스탯 기준 열 삭제 코드
# 1. 주스탯에 맞지 않는 능력치 칼럼 제거
# 2. INT 직업은 attack_power_total 제거, 비 INT 직업은 magic_power_total 제거
# 3. 공통적으로 제거할 컬럼: armor, speed, jump, equipment_level_decrease, max_hp_rate, max_mp_rate
# 4. 보존 컬럼: boss_damage_total, ignore_monster_armor_total, all_stat_total, damage_total

# 주요 스탯 관련 열만 따로 정의
stat_related_cols = [
    "str_total", "dex_total", "int_total", "luk_total",
    "max_hp_total", "max_mp_total",
    "attack_power_total", "magic_power_total",
    "armor_total", "speed_total", "jump_total",
    "equipment_level_decrease_total", "max_hp_rate_total", "max_mp_rate_total",
    "boss_damage_total", "ignore_monster_armor_total", "all_stat_total", "damage_total"
]

# 2. 직업별 주스탯 매핑
main_stat_map = {
    "STR": ["히어로", "아델", "소울마스터", "아란", "제로", "팔라딘", "다크나이트", "카이저",
            "데몬슬레이어", "미하일", "블래스터", "은월", "바이퍼", "스트라이커", "캐논마스터", "아크"],
    "DEX": ["윈드브레이커", "메르세데스", "보우마스터", "패스파인더", "신궁", "카인", "와일드헌터",
            "엔젤릭버스터", "캡틴", "메카닉"],
    "INT": ["비숍", "아크메이지(불,독)", "아크메이지(썬,콜)", "라라", "배틀메이지", "에반", "루미너스",
            "키네시스", "플레임위자드", "일리움"],
    "LUK": ["나이트워커", "섀도어", "나이트로드", "듀얼블레이더", "팬텀", "호영", "칼리", "카데나"],
    "HP": ["데몬어벤져"],
    "STR_DEX_LUK": ["제논"]
}
job_to_main_stat = {job: stat for stat, jobs in main_stat_map.items() for job in jobs}
df["main_stat_type"] = df["subclass"].map(job_to_main_stat)

# 3. 열별로 직업 조건 기반 결측 처리
# 공통 제거 대상
global_drop_cols = [
    "armor_total", "speed_total", "jump_total",
    "equipment_level_decrease_total", "max_hp_rate_total", "max_mp_rate_total"
]
for col in global_drop_cols:
    if col in df.columns:
        df.loc[:, col] = pd.NA

# 주스탯별로 유지 조건 처리
if "str_total" in df.columns:
    df.loc[~df["main_stat_type"].isin(["STR", "STR_DEX_LUK"]), "str_total"] = pd.NA
if "dex_total" in df.columns:
    df.loc[~df["main_stat_type"].isin(["DEX", "STR_DEX_LUK"]), "dex_total"] = pd.NA
if "int_total" in df.columns:
    df.loc[df["main_stat_type"] != "INT", "int_total"] = pd.NA
if "luk_total" in df.columns:
    df.loc[~df["main_stat_type"].isin(["LUK", "STR_DEX_LUK"]), "luk_total"] = pd.NA
if "max_hp_total" in df.columns:
    df.loc[df["main_stat_type"] != "HP", "max_hp_total"] = pd.NA
if "max_mp_total" in df.columns:
    df.loc[:, "max_mp_total"] = pd.NA  # 항상 제거 대상

# 공격력/마력 처리
if "attack_power_total" in df.columns:
    df.loc[df["main_stat_type"] == "INT", "attack_power_total"] = pd.NA
if "magic_power_total" in df.columns:
    df.loc[df["main_stat_type"] != "INT", "magic_power_total"] = pd.NA

<ipython-input-18-54b351321a26>:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<NA>' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = pd.NA
<ipython-input-18-54b351321a26>:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<NA>' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = pd.NA
<ipython-input-18-54b351321a26>:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<NA>' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:, col] = pd.NA
<ipython-input-18-54b351321a26>:40: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '<NA>' has dtype incompatible w

In [ ]:
# str_total, dex_total, int_total, luk_total, max_hp_total을 퉁쳐서 mainstat_total이라는 하나의 열로 만들기

import numpy as np

# 제논 직업 기준으로 3스탯 합산 적용
main_stats = df["subclass"]

# 제논 여부
is_zenon = main_stats == "제논"
non_zenon = ~is_zenon

# 각 열 타입 정리
cols_to_merge = ["str_total", "dex_total", "int_total", "luk_total", "max_hp_total"]
for col in cols_to_merge:
    if col in df.columns:
        df[col] = df[col].astype("float64")

# 1. 제논은 str + dex + luk 합산
df["mainstat_total"] = np.nan
df.loc[is_zenon, "mainstat_total"] = df.loc[is_zenon, ["str_total", "dex_total", "luk_total"]].sum(axis=1, skipna=True)

# 2. 그 외는 기존처럼 우선순위 기반 bfill
df.loc[non_zenon, "mainstat_total"] = df.loc[non_zenon, cols_to_merge].bfill(axis=1).iloc[:, 0]

# 3. 정리 후 필요 없는 열 제거
cols_to_drop = [
    "str_total", "dex_total", "int_total", "luk_total", "preset_number", "max_mp_total", "max_hp_total",
    "armor_total", "speed_total", "jump_total"
]
df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

In [ ]:
# attack_power_total과 magic_power_total을 하나의 열 power_total로 통합

# 수치형 결측치 처리를 위해 float64로 변환
df["attack_power_total"] = df["attack_power_total"].astype("float64")
df["magic_power_total"] = df["magic_power_total"].astype("float64")

# 두 열 중 결측이 아닌 값을 선택
df["power_total"] = df[["attack_power_total", "magic_power_total"]].bfill(axis=1).iloc[:, 0]

# 기존 열 삭제
df.drop(columns=["attack_power_total", "magic_power_total"], inplace=True)

# 불필요한 열 한 번 더 드롭
cols_to_drop = ["equipment_level_decrease_total", "max_hp_rate_total", "max_mp_rate_total"]
df.drop(columns=cols_to_drop, inplace=True)

### 장비 잠재능력 + 에디셔널 잠재능력
----
전략:
- 잠재능력: S-F등급 중 S-B 등급에 해당하는 옵션만 남겨두고, 나머지 옵션은 '기타'로 분류

In [ ]:
# 메모리 효율을 위해 벡터화 방식으로 등급 분류 처리(잠재능력)

# step 1: S~B 키워드 정의
grade_keywords = {
    "S": [
        "보스 몬스터 공격 시 데미지", "공격력 : +%", "마력 : +%", "올스탯 : +%", "몬스터 방어율 무시",
        "모든 스킬의 재사용 대기시간", "아이템 드롭률", "메소 획득량"
    ],
    "A": [
        "크리티컬 데미지"
    ],
    "B": [
        "공격력 : +32", "마력 : +32", "크리티컬 확률 :"
    ]
}

# step 2: 단일 스탯 % 패턴
stat_keywords = ["STR : +", "DEX : +", "INT : +", "LUK : +"]

# step 3: 직업별 예외 조건 처리용 사전
job_special = {
    "쓸만한 하이퍼 바디": {
        "S": {"데몬어벤져", "제논"},
        "B": {"비숍"}
    }
}

# step 4: 벡터화 방식으로 열별 등급 처리
def grade_vectorized(df, colname, subclass_col="subclass"):
    out = np.full(len(df), "기타", dtype=object)

    # 직업별 특수 옵션 우선 처리
    if "쓸만한 하이퍼 바디" in df[colname].astype(str).values.tolist():
        for grade, jobset in job_special["쓸만한 하이퍼 바디"].items():
            mask = df[colname].str.contains("쓸만한 하이퍼 바디", na=False) & df[subclass_col].isin(jobset)
            out[mask] = grade

    # 단일 스탯 % 증가는 S
    for stat in stat_keywords:
        mask = df[colname].str.contains(stat, na=False) & df[colname].str.contains("%", na=False)
        out[mask] = "S"

    # 일반 키워드 처리
    for grade, patterns in grade_keywords.items():
        for pat in patterns:
            mask = df[colname].str.contains(pat, na=False) & (out == "기타")
            out[mask] = grade

    return out

# 대상 열
potential_cols = [
    "potential_option_1", "potential_option_2", "potential_option_3",
]

# 등급화 열 생성
for col in potential_cols:
    df[f"{col}_grade"] = grade_vectorized(df, col)

# 요약 리스트 생성
df["main_pot_grade_summary"] = df[[
    "potential_option_1_grade", "potential_option_2_grade", "potential_option_3_grade"
]].values.tolist()

In [ ]:
# 메모리 효율을 위해 벡터화 방식으로 등급 분류 처리(에디셔널 잠재능력)

# (1) 에디셔널 잠재능력에서 S~B로 인정되는 키워드 정의
add_potential_grade_keywords = {
    "S": [
        "공격력 : +%", "마력 : +%", "크리티컬 데미지", "재사용 대기시간 감소",
        "캐릭터 기준 9레벨 당.*\+2",  # 정규표현식: +2 조건
    ],
    "A": [
        "보스 몬스터 공격 시 데미지", "데미지 : +%", "올스탯 : +%", "재사용 대기시간 감소",
        "HP : +%", "MP : +%", "캐릭터 기준 9레벨 당.*\+1"
    ],
    "B": [
        "크리티컬 확률", "공격력 : \+32", "마력 : \+32", "STR : \+32", "DEX : \+32", "INT : \+32", "LUK : \+32",
        "공격력 : \+\d+", "마력 : \+\d+", "STR : \+\d+", "DEX : \+\d+", "INT : \+\d+", "LUK : \+\d+"
    ]
}

# (2) 직업 예외 처리: 단일 스탯 % 증가 (제논 제외, 무기 제외는 생략)
def is_stat_percent_option(opt):
    return any(f"{stat} : +" in opt and "%" in opt for stat in ["STR", "DEX", "INT", "LUK"])

# (3) 직업 예외 처리: HP/MP % 증가는 데벤져만 인정
def is_hpmp_percent_option(opt):
    return ("HP : +%" in opt or "MP : +%" in opt)

# (4) 등급 분류 함수 (벡터화 방식용)
def add_grade_vectorized(df, colname):
    out = np.full(len(df), "기타", dtype=object)

    # 1. 단일 스탯 % 증가: 제논은 제외
    stat_mask = df[colname].str.contains("STR : +%", na=False) | \
                df[colname].str.contains("DEX : +%", na=False) | \
                df[colname].str.contains("INT : +%", na=False) | \
                df[colname].str.contains("LUK : +%", na=False)
    stat_mask &= (df["subclass"] != "제논")
    out[stat_mask] = "S"

    # 2. HP/MP % 증가 → 데몬어벤져만 인정
    hpmp_mask = df[colname].str.contains("HP : +%", na=False) | df[colname].str.contains("MP : +%", na=False)
    hpmp_mask &= df["subclass"] == "데몬어벤져"
    out[hpmp_mask] = "S"

    # 3. 키워드 기반 일반 등급화
    for grade, patterns in add_potential_grade_keywords.items():
        for pat in patterns:
            mask = df[colname].str.contains(pat, na=False, regex=True) & (out == "기타")
            out[mask] = grade

    return out

# 대상: 에디셔널 잠재능력 열
add_pot_cols = [
    "additional_potential_option_1", "additional_potential_option_2", "additional_potential_option_3"
]

# 에디셔널 등급화 열 생성
for col in add_pot_cols:
    df[f"{col}_grade"] = add_grade_vectorized(df, col)

# 에디셔널 요약 리스트 생성
df["add_pot_grade_summary"] = df[[
    "additional_potential_option_1_grade",
    "additional_potential_option_2_grade",
    "additional_potential_option_3_grade"
]].values.tolist()

In [ ]:
# 쓸모 없는 열 제거
cols_to_drop = [
    "potential_option_1", "potential_option_2", "potential_option_3",
    "additional_potential_option_1", "additional_potential_option_2", "additional_potential_option_3",
    "equipment_level_increase"
]

df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

In [ ]:
df.info() # 84 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063724 entries, 0 to 1063723
Data columns (total 84 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   nickname                             1063724 non-null  object 
 1   world                                1063724 non-null  object 
 2   level                                1063724 non-null  int64  
 3   job                                  1063724 non-null  object 
 4   subclass                             1063724 non-null  object 
 5   equipment_part                       1063724 non-null  object 
 6   equipment_slot                       1063724 non-null  object 
 7   item_name                            1063724 non-null  object 
 8   boss_damage_total                    1063724 non-null  int64  
 9   ignore_monster_armor_total           1063724 non-null  int64  
 10  all_stat_total                       1063724 non-null  int64  
 11

### 장비 특별 옵션(익셉셔널 강화)

In [ ]:
# 익셉셔널 강화의 경우, 최상위 컨텐츠를 도전할 수 있는 초고스펙 유저들만 이용 가능한 컨텐츠
# 1. 따라서, ['exceptional_upgrade']열에 대해 0.0 = False, 1.0 = True로 바꾸고(익셉셔널 강화 했는지 안했는지 정도만 보려고)
# 2. 나머지 열들은 모두 드롭

# 0.0 → False, 1.0 → True 변환
df["exceptional_upgrade"] = df["exceptional_upgrade"] == 1.0

# 나머지 익셉셔널 옵션 드롭
exceptional_cols_to_drop = [
    "str_exceptional", "dex_exceptional", "int_exceptional", "luk_exceptional",
    "max_hp_exceptional", "max_mp_exceptional",
    "attack_power_exceptional", "magic_power_exceptional"
]
df.drop(columns=[col for col in exceptional_cols_to_drop if col in df.columns], inplace=True)

In [ ]:
df.info() # 76 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063724 entries, 0 to 1063723
Data columns (total 76 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   nickname                             1063724 non-null  object 
 1   world                                1063724 non-null  object 
 2   level                                1063724 non-null  int64  
 3   job                                  1063724 non-null  object 
 4   subclass                             1063724 non-null  object 
 5   equipment_part                       1063724 non-null  object 
 6   equipment_slot                       1063724 non-null  object 
 7   item_name                            1063724 non-null  object 
 8   boss_damage_total                    1063724 non-null  int64  
 9   ignore_monster_armor_total           1063724 non-null  int64  
 10  all_stat_total                       1063724 non-null  int64  
 11

### 장비 추가 옵션

In [ ]:
# 직업별 주스탯 기준 열 삭제 코드
# 1. 주스탯에 맞지 않는 능력치 칼럼 제거
# 2. INT 직업은 attack_power_total 제거, 비 INT 직업은 magic_power_total 제거
# 3. 공통적으로 제거할 컬럼: armor, speed, jump, equipment_level_decrease, max_hp_rate, max_mp_rate
# 4. 보존 컬럼: boss_damage_total, ignore_monster_armor_total, all_stat_total, damage_total

# 주스탯 기반 벡터화 처리
def get_main_stat(subclass):
    for stat, jobs in main_stat_map.items():
        if subclass in jobs:

            return stat
    return None

main_stats = df["subclass"].map(get_main_stat)

# 주스탯이 아닌 경우만 NaN 처리
for stat_key, col in zip(["STR", "DEX", "INT", "LUK"],
                         ["str_add", "dex_add", "int_add", "luk_add"]):
    mask = ~((main_stats == stat_key) | ((main_stats == "STR,DEX,LUK") & (stat_key in ["STR", "DEX", "LUK"])))
    df.loc[mask, col] = np.nan

# HP 스탯 예외 처리: max_hp_add 유지 여부
mask_hp = ~((main_stats == "HP") | ((main_stats == "STR,DEX,LUK")))
df.loc[mask_hp, "max_hp_add"] = np.nan

# attack/magic 파워 제거 조건
df.loc[main_stats == "INT", "attack_power_add"] = np.nan
df.loc[(main_stats != "INT") & (main_stats != "STR,DEX,LUK"), "magic_power_add"] = np.nan

# 공통 제거 대상 (HP 관련 제외)
df.drop(columns=[
    "armor_add", "speed_add", "jump_add", "equipment_level_decrease_add", "max_mp_add"
], inplace=True)

In [ ]:
# str_add, dex_add, int_add, luk_add, max_hp_add를 퉁쳐서 mainstat_add라는 하나의 열로 만들기
# attack_power_add와 magic_power_add를 하나의 열 power_add로 통합

# 제논 여부 마스킹
is_zenon = df["subclass"] == "제논"
non_zenon = ~is_zenon

# 1. 제논: str + dex + luk 합산
df["mainstat_add"] = np.nan
df.loc[is_zenon, "mainstat_add"] = df.loc[is_zenon, ["str_add", "dex_add", "luk_add"]].sum(axis=1, skipna=True)

# 2. 비제논: 우선순위 bfill 방식
df.loc[non_zenon, "mainstat_add"] = df.loc[non_zenon, ["str_add", "dex_add", "int_add", "luk_add", "max_hp_add"]].bfill(axis=1).iloc[:, 0]

# 3. power_add 통합
df["power_add"] = df[["attack_power_add", "magic_power_add"]].bfill(axis=1).iloc[:, 0]

# 원본 열 제거
df.drop(columns=[
    "str_add", "dex_add", "int_add", "luk_add", "max_hp_add",
    "attack_power_add", "magic_power_add"
], inplace=True)

In [ ]:
df.info() # 66 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063724 entries, 0 to 1063723
Data columns (total 66 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   nickname                             1063724 non-null  object 
 1   world                                1063724 non-null  object 
 2   level                                1063724 non-null  int64  
 3   job                                  1063724 non-null  object 
 4   subclass                             1063724 non-null  object 
 5   equipment_part                       1063724 non-null  object 
 6   equipment_slot                       1063724 non-null  object 
 7   item_name                            1063724 non-null  object 
 8   boss_damage_total                    1063724 non-null  int64  
 9   ignore_monster_armor_total           1063724 non-null  int64  
 10  all_stat_total                       1063724 non-null  int64  
 11

### 장비 기타 옵션(주문서 강화)

In [ ]:
# 주스탯 매핑 그대로 재사용
main_stat_map = {
    "STR": ["히어로", "아델", "소울마스터", "아란", "제로", "팔라딘", "다크나이트", "카이저",
            "데몬슬레이어", "미하일", "블래스터", "은월", "바이퍼", "스트라이커", "캐논마스터", "아크"],
    "DEX": ["윈드브레이커", "메르세데스", "보우마스터", "패스파인더", "신궁", "카인", "와일드헌터",
            "엔젤릭버스터", "캡틴", "메카닉"],
    "INT": ["비숍", "아크메이지(불,독)", "아크메이지(썬,콜)", "라라", "배틀메이지", "에반", "루미너스",
            "키네시스", "플레임위자드", "일리움"],
    "LUK": ["나이트워커", "섀도어", "나이트로드", "듀얼블레이더", "팬텀", "호영", "칼리", "카데나"],
    "HP": ["데몬어벤져"],
    "STR,DEX,LUK": ["제논"]
}

# 주스탯 추출
def get_main_stat(subclass):
    for stat, jobs in main_stat_map.items():
        if subclass in jobs:
            return stat
    return None

main_stats = df["subclass"].map(get_main_stat)

# 주스탯별 제외할 열: STR~LUK
for stat_key, col in zip(["STR", "DEX", "INT", "LUK"],
                         ["str_etc", "dex_etc", "int_etc", "luk_etc"]):
    mask = ~((main_stats == stat_key) | ((main_stats == "STR,DEX,LUK") & (stat_key in ["STR", "DEX", "LUK"])))
    df.loc[mask, col] = np.nan

# max_hp_etc 처리: HP 주스탯인 경우만 유지
mask_max_hp = main_stats != "HP"
df.loc[mask_max_hp, "max_hp_etc"] = np.nan

# 공통적으로 제거할 열
df.drop(columns=[
    "max_mp_etc", "armor_etc", "speed_etc", "jump_etc"
], inplace=True)

In [ ]:
# str_add, dex_add, int_add, luk_add, max_hp_add를 퉁쳐서 mainstat_add라는 하나의 열로 만들기
# attack_power_add와 magic_power_add를 하나의 열 power_add로 통합

# 제논만 특수 처리: STR/DEX/LUK 3개를 더한 값으로 mainstat_etc 구성

# 벡터 기반 처리
mainstat_etc = np.full(len(df), np.nan)
is_zenon = df["subclass"] == "제논"

# 제논은 STR + DEX + LUK 더해서 넣기
mainstat_etc[is_zenon] = df.loc[is_zenon, ["str_etc", "dex_etc", "luk_etc"]].sum(axis=1, skipna=True)

# 나머지 직업군은 우선순위 채우기 (str → dex → int → luk → max_hp)
non_zenon = ~is_zenon
mainstat_etc[non_zenon] = df.loc[non_zenon, ["str_etc", "dex_etc", "int_etc", "luk_etc", "max_hp_etc"]].bfill(axis=1).iloc[:, 0]

# power_etc 통합 (attack or magic power)
df["mainstat_etc"] = mainstat_etc
df["power_etc"] = df[["attack_power_etc", "magic_power_etc"]].bfill(axis=1).iloc[:, 0]

# 원본 열 드롭
df.drop(columns=[
    "str_etc", "dex_etc", "int_etc", "luk_etc", "max_hp_etc",
    "attack_power_etc", "magic_power_etc"
], inplace=True)

In [ ]:
df.info() # 57 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063724 entries, 0 to 1063723
Data columns (total 57 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   nickname                             1063724 non-null  object 
 1   world                                1063724 non-null  object 
 2   level                                1063724 non-null  int64  
 3   job                                  1063724 non-null  object 
 4   subclass                             1063724 non-null  object 
 5   equipment_part                       1063724 non-null  object 
 6   equipment_slot                       1063724 non-null  object 
 7   item_name                            1063724 non-null  object 
 8   boss_damage_total                    1063724 non-null  int64  
 9   ignore_monster_armor_total           1063724 non-null  int64  
 10  all_stat_total                       1063724 non-null  int64  
 11

### 장비 스타포스 옵션

In [ ]:
# 주스탯 맵 재활용
main_stat_map = {
    "STR": ["히어로", "아델", "소울마스터", "아란", "제로", "팔라딘", "다크나이트", "카이저",
            "데몬슬레이어", "미하일", "블래스터", "은월", "바이퍼", "스트라이커", "캐논마스터", "아크"],
    "DEX": ["윈드브레이커", "메르세데스", "보우마스터", "패스파인더", "신궁", "카인", "와일드헌터",
            "엔젤릭버스터", "캡틴", "메카닉"],
    "INT": ["비숍", "아크메이지(불,독)", "아크메이지(썬,콜)", "라라", "배틀메이지", "에반", "루미너스",
            "키네시스", "플레임위자드", "일리움"],
    "LUK": ["나이트워커", "섀도어", "나이트로드", "듀얼블레이더", "팬텀", "호영", "칼리", "카데나"],
    "HP": ["데몬어벤져"],
    "STR,DEX,LUK": ["제논"]
}

def get_main_stat(subclass):
    for stat, jobs in main_stat_map.items():
        if subclass in jobs:
            return stat
    return None

main_stats = df["subclass"].map(get_main_stat)

# 주스탯 필드별 제거
for stat_key, col in zip(["STR", "DEX", "INT", "LUK"],
                         ["str_starforce", "dex_starforce", "int_starforce", "luk_starforce"]):
    mask = ~((main_stats == stat_key) | ((main_stats == "STR,DEX,LUK") & (stat_key in ["STR", "DEX", "LUK"])))
    df.loc[mask, col] = np.nan

# HP 주스탯인 경우만 max_hp 유지
df.loc[main_stats != "HP", "max_hp_starforce"] = np.nan

# 공통 제거 대상
drop_cols = ["max_mp_starforce", "armor_starforce", "speed_starforce", "jump_starforce"]
df.drop(columns=[col for col in drop_cols if col in df.columns], inplace=True)

In [ ]:
# 제논 여부 마스킹
is_zenon = df["subclass"] == "제논"
non_zenon = ~is_zenon

# 1. 제논: STR + DEX + LUK 합산
df["mainstat_starforce"] = np.nan
df.loc[is_zenon, "mainstat_starforce"] = df.loc[is_zenon, ["str_starforce", "dex_starforce", "luk_starforce"]].sum(axis=1, skipna=True)

# 2. 그 외는 우선순위 기반 bfill 방식
df.loc[non_zenon, "mainstat_starforce"] = df.loc[non_zenon, ["str_starforce", "dex_starforce", "int_starforce", "luk_starforce", "max_hp_starforce"]].bfill(axis=1).iloc[:, 0]

# 3. 공격력/마력 통합
df["power_starforce"] = df[["attack_power_starforce", "magic_power_starforce"]].bfill(axis=1).iloc[:, 0]

# 4. 원본 열 제거
df.drop(columns=[
    "str_starforce", "dex_starforce", "int_starforce", "luk_starforce", "max_hp_starforce",
    "attack_power_starforce", "magic_power_starforce"
], inplace=True)

In [ ]:
df.info() # 48 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063724 entries, 0 to 1063723
Data columns (total 48 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   nickname                             1063724 non-null  object 
 1   world                                1063724 non-null  object 
 2   level                                1063724 non-null  int64  
 3   job                                  1063724 non-null  object 
 4   subclass                             1063724 non-null  object 
 5   equipment_part                       1063724 non-null  object 
 6   equipment_slot                       1063724 non-null  object 
 7   item_name                            1063724 non-null  object 
 8   boss_damage_total                    1063724 non-null  int64  
 9   ignore_monster_armor_total           1063724 non-null  int64  
 10  all_stat_total                       1063724 non-null  int64  
 11

### 기타

In [ ]:
# 기타 불필요한 옵션도 최종적으로 드롭!
cols_to_drop = [
    "world",
    "level",
    "job",
    "equipment_part",
    "growth_exp",
    "growth_level",
    "scroll_upgrade",
    "cuttable_count",
    "golden_hammer_flag",
    "scroll_resilience_count",
    "scroll_upgradeable_count",
    "soul_name",
    "soul_option"
]

# 실제 존재하는 열만 필터링 후 삭제
df.drop(columns=[col for col in cols_to_drop if col in df.columns], inplace=True)

In [ ]:
df.info() # 최종 칼럼 개수: 35개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063724 entries, 0 to 1063723
Data columns (total 35 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   nickname                             1063724 non-null  object 
 1   subclass                             1063724 non-null  object 
 2   equipment_slot                       1063724 non-null  object 
 3   item_name                            1063724 non-null  object 
 4   boss_damage_total                    1063724 non-null  int64  
 5   ignore_monster_armor_total           1063724 non-null  int64  
 6   all_stat_total                       1063724 non-null  int64  
 7   damage_total                         1063724 non-null  int64  
 8   potential_option_grade               882109 non-null   object 
 9   additional_potential_option_grade    858435 non-null   object 
 10  exceptional_upgrade                  1063724 non-null  bool   
 11

In [ ]:
df['equipment_slot'].unique()

array(['모자', '상의', '하의', '신발', '장갑', '망토', '보조무기', '무기', '훈장', '벨트',
       '어깨장식', '포켓 아이템', '기계 심장', '뱃지', '엠블렘', '얼굴장식', '눈장식', '귀고리',
       '반지1', '반지2', '반지3', '반지4', '펜던트', '펜던트2'], dtype=object)

In [ ]:
df['equipment_part'].unique() # 무기 종류를 상세히 구분할 필요는 없음. 제거 완료

array(['모자', '상의', '하의', '신발', '장갑', '망토', '무기 전송장치', '체인', '훈장', '벨트',
       '어깨장식', '포켓 아이템', '기계 심장', '뱃지', '엠블렘', '얼굴장식', '눈장식', '귀고리', '반지',
       '펜던트', '한벌옷', '블레이드', '단검', '선추', '부채', '헥스시커', '차크람', '아대', '부적',
       '한손둔기', '보석', '두손둔기', '카드', '케인', '방패', '단검용 검집', '한손검', '컨트롤러',
       '에너지소드', '화살깃', '활', '활골무', '석궁', '렐릭', '에인션트 보우', '화살촉', '마법화살',
       '듀얼 보우건', '웨폰 벨트', '브레스 슈터', '소울실드', '두손검', '포스실드', '한손도끼',
       '데스페라도', '장약', '건틀렛 리볼버', '브레이슬릿', '튜너', '무게추', '폴암', '쇠사슬', '창',
       '로자리오', '메달', '두손도끼', '대검', '태도', '용의 정수', '너클', '매그넘', '건',
       '패스 오브 어비스', '소울링', '소울슈터', '여우구슬', '리스트밴드', '화약통', '핸드캐논', '조준기',
       '마도서', '스태프', '완드', '마법구슬', '문서', '매직윙', '매직 건틀렛', '체스피스',
       'ESP 리미터', '노리개', '샤이닝 로드', '오브'], dtype=object)

In [ ]:
# 저장
save_path = "/content/drive/MyDrive/item_preprocessed.csv" # 저장 경로 수정해주세요
df.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"CSV 저장 완료: {save_path}")

CSV 저장 완료: /content/drive/MyDrive/item_preprocessed.csv


## 결측치 처리

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/item_preprocessed.csv')

In [ ]:
df.isna().sum()

# 결측치 있는 칼럼 개수 총 2개

,0
nickname,0
subclass,0
equipment_slot,0
item_name,0
boss_damage_total,0
ignore_monster_armor_total,0
all_stat_total,0
damage_total,0
potential_option_grade,181615
additional_potential_option_grade,205289


### 잠재능력
- potential_option_grade: 181615

In [ ]:
df['potential_option_grade'].value_counts(dropna=False)

,count
potential_option_grade,
레전드리,685517
NaN,181615
유니크,151589
에픽,43172
레어,1831


In [ ]:
# 아이템의 잠재능력, 에디셔널 잠재능력에서 결측치 다수 발견
# 먼저 잠재능력이 붙을 수 없는 아이템 걸러내기

df_null = df[df['potential_option_grade'].isna()]
df_null['equipment_slot'].value_counts()

,count
equipment_slot,
뱃지,45848
포켓 아이템,45837
훈장,45619
반지1,31545
반지4,1893
펜던트2,1881
펜던트,1476
반지2,988
반지3,741


In [ ]:
# 잠재옵션이 아예 존재하지 않는 장비 부위 목록
non_potential_slots = ['포켓 아이템', '뱃지', '훈장']

# 해당 부위에 대해 'potential_option_grade'가 결측치인 경우 'None'으로 대체
df.loc[
    (df['equipment_slot'].isin(non_potential_slots)) &
    (df['potential_option_grade'].isna()),
    'potential_option_grade'
] = 'None'

In [ ]:
# 잠재옵션이 붙을 수 없는 '특수 능력 반지' + 기타 반지 종류 구분하기

# (1) 반지1
df_null[df_null['equipment_slot']=='반지1']['item_name'].value_counts()

,count
item_name,
리스트레인트 링,17836
컨티뉴어스 링,9665
웨폰퍼프 - I링,1351
웨폰퍼프 - D링,884
웨폰퍼프 - S링,790
웨폰퍼프 - L링,299
어비스 헌터스 링,183
리스크테이커 링,143
링 오브 썸,123


In [ ]:
# (2) 반지2
df_null[df_null['equipment_slot']=='반지2']['item_name'].value_counts()

,count
item_name,
컨티뉴어스 링,408
어드벤처 딥다크 크리티컬 링,267
리스트레인트 링,161
웨폰퍼프 - I링,40
어비스 헌터스 링,33
웨폰퍼프 - D링,20
웨폰퍼프 - S링,15
챌린저스 컨티뉴어스 링,12
가디언 엔젤 링,8


In [ ]:
# (3) 반지3
df_null[df_null['equipment_slot']=='반지3']['item_name'].value_counts()

,count
item_name,
컨티뉴어스 링,269
어드벤처 딥다크 크리티컬 링,236
리스트레인트 링,124
어비스 헌터스 링,33
웨폰퍼프 - I링,22
웨폰퍼프 - D링,15
챌린저스 컨티뉴어스 링,10
웨폰퍼프 - S링,7
가디언 엔젤 링,4


In [ ]:
# (4) 반지4
df_null[df_null['equipment_slot']=='반지4']['item_name'].value_counts()

,count
item_name,
컨티뉴어스 링,931
어드벤처 딥다크 크리티컬 링,374
리스트레인트 링,320
어비스 헌터스 링,94
웨폰퍼프 - I링,48
챌린저스 컨티뉴어스 링,37
웨폰퍼프 - D링,29
웨폰퍼프 - L링,12
웨폰퍼프 - S링,12


In [ ]:
# 1. 잠재옵션이 부여되지 않는 반지 이름 목록
non_potential_rings = [
    # 시드링 (특수 능력 반지)
    "리스트레인트 링", "리스크테이커 링", "크라이시스 - HM링", "컨티뉴어스 링",
    "웨폰퍼프 - S링", "웨폰퍼프 - D링", "웨폰퍼프 - I링", "웨폰퍼프 - L링",
    "레벨퍼프 - S링", "레벨퍼프 - D링", "레벨퍼프 - I링", "레벨퍼프 - L링",
    "링 오브 썸", "듀라빌리티 링", "얼티메이덤 링", "크리데미지 링",
    "크라이시스 - H링", "크라이시스 - M링", "크리디펜스 링", "스위프트 링",
    "헬스컷 링", "마나컷 링", "리밋 링", "실드스와프 링",
    "크리쉬프트 링", "스탠스쉬프트 링", "타워인헨스 링", "오버패스 링",
    "리플렉티브 링", "버든리프트 링", "리커버디펜스 링", "리커버스탠스 링",
    "챌린저스 컨티뉴어스 링", "챌린저스 리스트레인트 링",

    # 일반 이벤트/기타 반지
    "링 오브 페어리퀸", "이피아의 반지", "플래티넘 크로스 링",
    "브론즈 크로스 링", "실버 크로스 링", "골드 크로스 링",
    "엔젤릭 블레스", "다크 엔젤릭 블레스", "화이트 엔젤릭 블레스",
    "시그너스의 코히누르", "앱솔루트 링", "퍼스트 링",
    "오로라 링", "오닉스 링", "어비스 헌터스 링",
    "어드벤처 크리티컬 링", "다크 어드벤처 크리티컬 링",
    "어드벤처 딥다크 크리티컬 링", "어드벤처 다크 크리티컬 링",
    "에피네아의 반지", "모험가의 매직컬 링", "날렵한 백발백중의 링Ⅰ",
    "날쌘돌이의 링Ⅲ", "매지션링"
]

# 2. 반지 부위로 제한
ring_slots = ['반지1', '반지2', '반지3', '반지4']

# 3. 해당 조건에 해당하는 장비에 대해 잠재옵션 관련 결측치 'None'으로 대체
potential_cols = ['potential_option_grade']

for col in potential_cols:
    df.loc[
        (df['equipment_slot'].isin(ring_slots)) &
        (df['item_name'].isin(non_potential_rings)) &
        (df[col].isna()),
        col
    ] = 'None'

In [ ]:
# 몇 개나 채워졌는지 확인
df_null = df[df['potential_option_grade'].isna()]
df_null['equipment_slot'].value_counts()

# 반지1: 31174 -> 1
# 반지2: 986 -> 18
# 반지3: 740 -> 12
# 반지4: 1889 -> 22

,count
equipment_slot,
펜던트2,1881
펜던트,1476
모자,689
상의,620
하의,606
신발,559
장갑,538
망토,511
어깨장식,481


In [ ]:
# 남은 반지 확인

# 반지1
df_null[df_null['equipment_slot']=='반지1']['item_name'].value_counts()

,count
item_name,
SS급 마스터 쥬얼링,1


In [ ]:
# 반지2
df_null[df_null['equipment_slot']=='반지2']['item_name'].value_counts()

,count
item_name,
가디언 엔젤 링,8
여명의 가디언 엔젤 링,5
실버블라썸 링,1
고귀한 이피아의 반지,1
SS급 마스터 쥬얼링,1
거대한 공포,1
라이징 썬 링,1


In [ ]:
# 반지3
df_null[df_null['equipment_slot']=='반지3']['item_name'].value_counts()

,count
item_name,
가디언 엔젤 링,4
마이스터링,2
실버블라썸 링,2
히어로즈 프레셔스 링,1
여명의 가디언 엔젤 링,1
고귀한 이피아의 반지,1
거대한 공포,1


In [ ]:
# 반지4
df_null[df_null['equipment_slot']=='반지4']['item_name'].value_counts()

,count
item_name,
가디언 엔젤 링,11
여명의 가디언 엔젤 링,5
실버블라썸 링,4
10주년 블랙 메이플링,1
거대한 공포,1


In [ ]:
# 잠재능력 결측치인 애들 중, 반지 슬롯 1~4 중 하나라도 포함된 캐릭터 추출
ring_null_users = df[
    (df['potential_option_grade'].isna()) &
    (df['equipment_slot'].isin(['반지1', '반지2', '반지3', '반지4']))
]['nickname'].unique()

slot_counts_df = df[df['nickname'].isin(ring_null_users)] \
    .groupby('nickname')['equipment_slot'] \
    .nunique() \
    .reset_index(name='slot_count') \
    .sort_values(by='slot_count', ascending=False) \
    .reset_index(drop=True)

slot_counts_df

,nickname,slot_count
0,1979WH,24
1,래나민,24
2,러항,24
3,김은작,24
4,동띰,24
5,옛날곡,24
6,위잼,24
7,정요,24
8,초끈,24
9,카라라알랄리,24


In [ ]:
# 단순히 반지에 잠재능력을 부여하지 않았다, 아이템 슬롯 몇 개가 빈다는 이유로
# 특정 유저를 비활성화 유저라고 판단하기 어려웠음..

# 따라서 잠재능력을 붙일 수 있는 반지임에도 붙이지 않은 아이템들에 대해서는 'Not Granted'로 결측치를 채울 예정

# 잠재능력을 붙일 수 있는 반지 슬롯 리스트
potential_rings = ['반지1', '반지2', '반지3', '반지4']

# 해당 조건을 만족하는 경우에만 'Not Granted'로 채움
df.loc[
    (df['equipment_slot'].isin(potential_rings)) &
    (df['potential_option_grade'].isna()),
    'potential_option_grade'
] = 'Not Granted'

In [ ]:
df.isna().sum()

,0
nickname,0
subclass,0
equipment_slot,0
item_name,0
boss_damage_total,0
ignore_monster_armor_total,0
all_stat_total,0
damage_total,0
potential_option_grade,9144
additional_potential_option_grade,205289


In [ ]:
df_null = df[df['potential_option_grade'].isna()]
df_null['equipment_slot'].value_counts()

,count
equipment_slot,
펜던트2,1881
펜던트,1476
모자,689
상의,620
하의,606
신발,559
장갑,538
망토,511
어깨장식,481


In [ ]:
df_null[df_null['equipment_slot']=='펜던트2']['item_name'].value_counts()

,count
item_name,
정령의 펜던트,806
준비된 정령의 펜던트,562
그리드 펜던트,370
매커네이터 펜던트,82
스페셜 정령의 펜던트,45
데이브레이크 펜던트,6
교양인의 펜던트,2
혼테일의 목걸이,2
사랑의 족쇄,1


In [ ]:
# 모든 종류의 아이템에 대해 이렇게 일일이 잠재능력이 붙는 아이템인지 아닌지 확인하기 어렵다고 판단
# 잠재가 '한 번도' 붙은 적 없는 아이템 목록을 추출하고, 그 정보를 기반으로 결측치 처리
# 잠재가 '한 번도' 붙은 적 없음 = 'None' / 붙을 수 있는데도 붙이지 않았음 = 'Not Granted'로 처리

# 각 item_name별로 잠재능력 등급이 존재한 적 있는지 확인
item_potential_check = df.groupby('item_name')['potential_option_grade'] \
    .apply(lambda x: x.notna().sum()) \
    .reset_index(name='count_with_potential')

# 0이면 잠재 붙은 사례가 한 번도 없음 → 잠재 불가능 or 부여 안 함
no_potential_items = item_potential_check[item_potential_check['count_with_potential'] == 0]
no_potential_items['item_name'].unique()

array(['10주년 블랙 슈트', '10주년 화이트 치클', '10주년 화이트 펜던트', 'D베이직 웨폰 벨트', '가니어',
       '가죽 샌들', '가죽 핸드백', '갈색 도로시 구두', '강인함의샤이니로얄숄더', '개런드 피스톨', '검',
       '곡괭이', '구름 로브', '군밤장수 모자', '그린 아르나햇', '그린 아벨린', '그린 아벨린 치마',
       '그린 하드래더 부츠', '긍지의 소울실드', '낡은 글라디우스', '노바 리카온 부츠', '노바 히아데스 벨트',
       '돼지치기 막대', '드라카즈 군화', '드라카즈 제복', '드래곤 퍼플 슬레브', '드래곤테일 메이지케이프',
       '라이징 썬 페이스페인팅', '라이트 오브', '라즐리 7형', '라피스 7형', '레드 아이젠', '레이븐혼 차크람',
       '레이븐혼 체이서햇', '로얄 반 레온 메이지부츠', '루디브리엄 케이프', '마노 선추', '마지 카르트',
       '말랑말랑한 신발', '매그넘', '머리 위에 떡 두개', '메이플 3000일 망토', '메이플 리프',
       '메이플 모자 (2단계)', '메이플 모자 (4단계)', '메이플 모자 (기본)', '메이플 베릴 글러브',
       '메이플 베릴 리센느', '메이플 베릴 버클', '메이플 베릴 베레모', '메이플 베릴 숄더', '메이플 베릴 클록',
       '메이플 소드', '메탈실버 이어링', '무명도', '물컹물컹한 신발', '반반 투구', '베이직 매직윙',
       '봉인된 제네시스 데스페라도', '봉인된 제네시스 듀얼보우건', '봉인된 제네시스 매직 건틀렛',
       '봉인된 제네시스 보우', '봉인된 제네시스 샤이닝로드', '봉인된 제네시스 세이버', '봉인된 제네시스 스태프',
       '봉인된 제네시스 에너지체인', '봉인된 제네시스 에인션트 보우', '봉인된 제네시스 엑스',
       '봉인된 제네시스 엘라하', '봉인된 제네시스 

In [ ]:
# 잠재 없는 아이템 이름들
no_potential_item_list = no_potential_items['item_name'].tolist()

# 해당 아이템의 잠재 결측치 → 'None'
df.loc[
    (df['item_name'].isin(no_potential_item_list)) &
    (df['potential_option_grade'].isna()),
    'potential_option_grade'
] = 'None'

# 반대로, 잠재는 가능한데 안 붙인 애들 → 'Not Granted'
df.loc[
    (~df['item_name'].isin(no_potential_item_list)) &
    (df['potential_option_grade'].isna()),
    'potential_option_grade'
] = 'Not Granted'

In [ ]:
df.isna().sum()

,0
nickname,0
subclass,0
equipment_slot,0
item_name,0
boss_damage_total,0
ignore_monster_armor_total,0
all_stat_total,0
damage_total,0
potential_option_grade,0
additional_potential_option_grade,205289


### 에디셔널 잠재능력
- additional_potential_option_grade: 205289

In [ ]:
# 에디셔널 잠재능력에 대해서도 마찬가지로 진행!

# additional 잠재가 존재한 적이 있는 아이템만 체크
add_potential_check = df.groupby('item_name')['additional_potential_option_grade'] \
    .apply(lambda x: x.notna().sum()) \
    .reset_index(name='count_with_additional')

# 에디셔널이 단 한 번도 없었던 아이템들
no_add_potential_items = add_potential_check[add_potential_check['count_with_additional'] == 0]

# 리스트로 추출
no_add_potential_item_list = no_add_potential_items['item_name'].tolist()
no_add_potential_item_list

['10주년 블랙 슈트',
 '10주년 화이트 치클',
 '10주년 화이트 펜던트',
 '2022 싸전귀 골드',
 '2024 싸전귀 골드',
 '2024 싸전귀 레전드',
 'BLACK',
 'Black Heaven Lover',
 'DAVE THE DIVER',
 'DREAMER',
 'D베이직 웨폰 벨트',
 'HYPER BURNING',
 'HYPER BURNING MAX',
 'MILESTONE',
 'NEW AGE',
 'NEXT',
 'RED 기념 용사의 뱃지',
 'T-boy의 모니터',
 'V 페스티벌 뱃지',
 '★15번가 셀럽★',
 '☆메이프릴 아일랜드☆',
 '♠블루밍 메이플♠',
 '♣메이플 모멘트리♣',
 '가니어',
 '가죽 샌들',
 '가죽 핸드백',
 '갈색 도로시 구두',
 '감시자를 해방한 자',
 '강인함의 블루 버클 벨트',
 '강인함의샤이니로얄숄더',
 '개런드 피스톨',
 '검',
 '검은 천국의 대적자',
 '검은색 세라프의 망토',
 '검의 주인을 기다리며',
 '곡괭이',
 '골드 버거넷 헬름',
 '공허를 극복한 자',
 '광휘의 날개',
 '구름 로브',
 '구원은 그대의 손아귀에',
 '군밤장수 모자',
 '귀살대',
 '그린 아르나햇',
 '그린 아벨린',
 '그린 아벨린 치마',
 '그린 하드래더 부츠',
 '근성 하나로 살아가는 자',
 '근원을 파헤친 자',
 '금요일엔 역시 몬스터파크',
 '긍지의 소울실드',
 '꿈나무',
 '꿈의 오케스트라',
 '나는야 럭키가이★',
 '나비의 꿈',
 '낙원에 도달한 사람',
 '날렵한 백발백중의 링Ⅰ',
 '날쌘돌이의 링Ⅲ',
 '날카로운 그린 레더 벨트',
 '낡은 글라디우스',
 '내가 바로 액션 히어로',
 '내가 제일 잘 올라',
 '너의 목소리를 들려줘 뱃지',
 '네크로 매지션글러브',
 '네크로 매직 건틀렛',
 '네크로 센티널슈즈',
 '네크로 스키퍼슈트',
 '노가다 목장갑',
 '노련한 사냥꾼의 훈장',
 '노바 리카온 부츠',
 '노바 

In [ ]:
# 에디셔널 불가능 (한 번도 부여된 적 없는 아이템)
df.loc[
    (df['item_name'].isin(no_add_potential_item_list)) &
    (df['additional_potential_option_grade'].isna()),
    'additional_potential_option_grade'
] = 'None'

# 에디셔널 부여 가능하지만 안 붙인 상태
df.loc[
    (~df['item_name'].isin(no_add_potential_item_list)) &
    (df['additional_potential_option_grade'].isna()),
    'additional_potential_option_grade'
] = 'Not Granted'

In [ ]:
# '잠재/에디셔널 부여 상태 요약' 컬럼 만들기 <- 혹시 필요할까 만들어둠!
df['potential_status'] = df['potential_option_grade'] + ' / ' + df['additional_potential_option_grade']
df['potential_status']

,potential_status
0,레전드리 / 레전드리
1,레전드리 / 레전드리
2,레전드리 / 레전드리
3,레전드리 / 레전드리
4,레전드리 / 레전드리
...,...
1063719,None / None
1063720,유니크 / 에픽
1063721,None / None
1063722,레전드리 / 유니크


In [ ]:
df.isna().sum()

,0
nickname,0
subclass,0
equipment_slot,0
item_name,0
boss_damage_total,0
ignore_monster_armor_total,0
all_stat_total,0
damage_total,0
potential_option_grade,0
additional_potential_option_grade,0


In [ ]:
# 중간 저장

save_path = "/content/drive/MyDrive/item_preprocessed3.csv" # 저장 경로 수정해주세요
df.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"CSV 저장 완료: {save_path}")

CSV 저장 완료: /content/drive/MyDrive/item_preprocessed3.csv


## 문자형/리스트형 칼럼에 대한 인코딩

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/item_preprocessed3.csv')

In [ ]:
df.info()

# nickname: 어차피 모델 학습시킬 때 제외할 것!
# subclass: 직업
# equipment_slot: 아이템 종류
# item_name: 말 그대로 아이템명
# potential_option_grade
# additional_potential_option_grade
# starforce_scroll_flag
# main_stat_type
# potential_option_1_grade
# potential_option_2_grade
# potential_option_3_grade
# main_pot_grade_summary
# additional_potential_option_1_grade
# additional_potential_option_2_grade
# additional_potential_option_3_grade
# add_pot_grade_summary
# potential_status

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063724 entries, 0 to 1063723
Data columns (total 36 columns):
 #   Column                               Non-Null Count    Dtype  
---  ------                               --------------    -----  
 0   nickname                             1063724 non-null  object 
 1   subclass                             1063724 non-null  object 
 2   equipment_slot                       1063724 non-null  object 
 3   item_name                            1063724 non-null  object 
 4   boss_damage_total                    1063724 non-null  int64  
 5   ignore_monster_armor_total           1063724 non-null  int64  
 6   all_stat_total                       1063724 non-null  int64  
 7   damage_total                         1063724 non-null  int64  
 8   potential_option_grade               1063724 non-null  object 
 9   additional_potential_option_grade    1063724 non-null  object 
 10  exceptional_upgrade                  1063724 non-null  bool   
 11

In [ ]:
# 인코딩 대상 문자형 열 목록
object_columns = [
    'subclass', 'equipment_slot', 'item_name',
    'potential_option_grade', 'additional_potential_option_grade',
    'starforce_scroll_flag', 'main_stat_type',
    'potential_option_1_grade', 'potential_option_2_grade', 'potential_option_3_grade',
    'main_pot_grade_summary',
    'additional_potential_option_1_grade', 'additional_potential_option_2_grade', 'additional_potential_option_3_grade',
    'add_pot_grade_summary', 'potential_status'
]

# 각 컬럼별로 unique 값 수 + 값 리스트 출력
for col in object_columns:
    unique_vals = df[col].dropna().unique()
    print(f"\n📌 [{col}]")
    print(f" - 고유값 개수: {len(unique_vals)}")
    print(f" - 고유값 목록: {unique_vals if len(unique_vals) <= 20 else list(unique_vals[:20]) + ['...']}")


📌 [subclass]
 - 고유값 개수: 46
 - 고유값 목록: ['카데나', '듀얼블레이더', '호영', '칼리', '나이트로드', '나이트워커', '팬텀', '섀도어', '제논', '보우마스터', '신궁', '패스파인더', '윈드브레이커', '와일드헌터', '메르세데스', '카인', '미하일', '소울마스터', '데몬슬레이어', '데몬어벤져', '...']

📌 [equipment_slot]
 - 고유값 개수: 24
 - 고유값 목록: ['모자', '상의', '하의', '신발', '장갑', '망토', '보조무기', '무기', '훈장', '벨트', '어깨장식', '포켓 아이템', '기계 심장', '뱃지', '엠블렘', '얼굴장식', '눈장식', '귀고리', '반지1', '반지2', '...']

📌 [item_name]
 - 고유값 개수: 1173
 - 고유값 목록: ['에테르넬 시프반다나', '에테르넬 시프셔츠', '에테르넬 시프팬츠', '아케인셰이드 시프슈즈', '아케인셰이드 시프글러브', '아케인셰이드 시프케이프', '트랜스미터 type:A', '제네시스 체인', '칠요의 몬스터파커', '몽환의 벨트', '에테르넬 시프숄더', '저주받은 황의 마도서', '컴플리트 언더컨트롤', '창세의 뱃지', '미트라의 분노 : 도적', '루즈 컨트롤 머신 마크', '마력이 깃든 안대', '커맨더 포스 이어링', '리스트레인트 링', '여명의 가디언 엔젤 링', '...']

📌 [potential_option_grade]
 - 고유값 개수: 6
 - 고유값 목록: ['레전드리' 'None' '유니크' '에픽' '레어' 'Not Granted']

📌 [additional_potential_option_grade]
 - 고유값 개수: 6
 - 고유값 목록: ['레전드리' 'None' '에픽' '유니크' '레어' 'Not Granted']

📌 [starforce_scroll_flag]
 - 고유값 개수: 2
 - 고유값 목록: ['미사용' '사용']

📌 [ma

In [ ]:
# 결과 저장용 딕셔너리
from collections import defaultdict

slot_item_summary = defaultdict(dict)

for slot in df['equipment_slot'].dropna().unique():
    items = df[df['equipment_slot'] == slot]['item_name'].dropna().unique()
    slot_item_summary[slot]['count'] = len(items)
    slot_item_summary[slot]['examples'] = list(items[:10]) + (['...'] if len(items) > 10 else [])

# 보기 좋게 출력
for slot, info in slot_item_summary.items():
    print(f"\n📦 [{slot}]")
    print(f"- 고유 아이템 수: {info['count']}")
    print(f"- 예시: {info['examples']}")


📦 [모자]
- 고유 아이템 수: 76
- 예시: ['에테르넬 시프반다나', '하이네스 어새신보닛', '아케인셰이드 시프햇', '앱솔랩스 시프캡', '카오스 벨룸의 헬름', '도전자의 모자', '벨룸의 헬름', 'T-boy의 모니터', '레이븐혼 체이서햇', '군밤장수 모자', '...']

📦 [상의]
- 고유 아이템 수: 49
- 예시: ['에테르넬 시프셔츠', '이글아이 어새신셔츠', '도전자의 상의', '앱솔랩스 시프슈트', '카오스 핑크빈 슈트', '아케인셰이드 시프슈트', '레드 카티나스', '이글아이 원더러코트', '에테르넬 파이렛코트', '10주년 블랙 슈트', '...']

📦 [하의]
- 고유 아이템 수: 16
- 예시: ['에테르넬 시프팬츠', '트릭스터 어새신팬츠', '도전자의 하의', '트릭스터 원더러팬츠', '에테르넬 파이렛팬츠', '에테르넬 아처팬츠', '트릭스터 레인져팬츠', '그린 아벨린 치마', '아이스 진', '트릭스터 워리어팬츠', '...']

📦 [신발]
- 고유 아이템 수: 67
- 예시: ['아케인셰이드 시프슈즈', '에테르넬 시프슈즈', '앱솔랩스 시프슈즈', '타일런트 리카온 부츠', '도전자의 신발', '무스펠 로그슈즈', '다크 피레타부츠', '레이븐혼 체이서부츠', '레드 피레타부츠', '노바 리카온 부츠', '...']

📦 [장갑]
- 고유 아이템 수: 78
- 예시: ['아케인셰이드 시프글러브', '에테르넬 시프글러브', '앱솔랩스 시프글러브', '도전자의 장갑', '펜살리르 체이서글러브', '무스펠 로그글러브', '이클레틱 루바브', '튼튼한 기계 장갑', '레이븐혼 체이서글러브', '다크 아날린', '...']

📦 [망토]
- 고유 아이템 수: 48
- 예시: ['아케인셰이드 시프케이프', '에테르넬 시프케이프', '앱솔랩스 시프케이프', '도전자의 망토', '타일런트 리카온 클록', '레이븐혼 체이서케이프', '분노한 자쿰의 망토', '타일런트 알테어 클록', '앱솔랩스 파이렛케이프', '아케인

In [ ]:
# item_name 칼럼의 고유값 더 줄여보기
# 무기, 모자, 상의, 하의, 장갑, 신발, 망토, 어깨장식: item_name의 앞부분에서 세트명만 추출하도록 함
  # 도전자 / 파프니르 / 앱솔랩스 / 아케인셰이드 / 에테르넬 / 기타 / (무기만) 봉인된 제네시스 / (무기만)제네시스
# 보조무기: 그냥 '보조무기'로 사용
# 훈장: 그냥 '훈장'으로 사용
# 기계심장: 칠흑 셋 / 기타로 구분
# 벨트: 보스 셋 / 칠흑 셋 / 기타로 구분
# 반지: 시드링 / 이벤트 반지 / 기타 구분
# 포켓 아이템: 보스 셋 / 칠흑 셋 / 기타로 구분
# 뱃지: 보스 셋 / 칠흑 셋 / 기타로 구분
# 엠블렘: 칠흑 셋 / 기타로 구분
# 얼굴장식: 보스 셋 / 여명 셋 / 칠흑 셋 / 기타로 구분
# 눈장식: 보스 셋 / 칠흑 셋 / 기타로 구분
# 귀고리: 보스 셋 / 여명 셋 / 칠흑 셋 / 기타로 구분
# 펜던트: 보스 셋 / 여명 셋 / 칠흑 셋 / 기타로 구분

In [ ]:
def get_item_group(row):
    slot = row['equipment_slot']
    name = row['item_name']

    # Null 방지
    if pd.isna(name):
        return '기타'

    # 세트 장비
    set_slots = ['무기', '모자', '상의', '하의', '장갑', '신발', '망토', '어깨장식']
    if slot in set_slots:
        if slot == '무기':
            if '봉인된 제네시스' in name:
                return '봉인된 제네시스'
            elif '제네시스' in name:
                return '제네시스'
        if '도전자의' in name:
            return '도전자'
        elif '파프니르' in name:
            return '파프니르'
        elif '앱솔랩스' in name:
            return '앱솔랩스'
        elif '아케인셰이드' in name:
            return '아케인셰이드'
        elif '에테르넬' in name:
            return '에테르넬'
        return '기타'

    # 보조무기/훈장
    if slot == '보조무기':
        return '보조무기'
    if slot == '훈장':
        return '훈장'

    # 기계 심장
    if slot == '기계 심장':
        if '블랙 하트' in name or '컴플리트 언더컨트롤' in name:
            return '칠흑'
        return '기타'

    # 벨트
    if slot == '벨트':
        if '골든 클로버 벨트' in name or '분노한 자쿰의 벨트' in name:
            return '보스 장신구'
        elif '몽환의 벨트' in name:
            return '칠흑'
        return '기타'

    # 반지
    seed_keywords = [
        '리스트레인트 링', '리스크테이커 링', '크라이시스 - HM링', '컨티뉴어스 링',
        '웨폰퍼프', '레벨퍼프', '링 오브', '듀라빌리티 링', '얼티메이덤 링',
        '크리데미지 링', '크라이시스 - H링', '크라이시스 - M링', '크리디펜스 링',
        '스위프트 링', '헬스컷 링', '마나컷 링', '리밋 링', '실드스와프 링',
        '크리쉬프트 링', '스탠스쉬프트 링', '타워인헨스 링', '오버패스 링',
        '리플렉티브 링', '버든리프트 링', '리커버디펜스 링', '리커버스탠스 링',
        '챌린저스 컨티뉴어스 링', '챌린저스 리스트레인트 링'
    ]
    event_keywords = [
        '오닉스 링', '벤젼스 링', '코스모스 링', 'SS급', '결속의 반지',
        '크리티컬링', '카오스 링', '테네브리스', '글로리온 링', '어웨이크 링',
        '이터널 플레임 링', '어비스 헌터스 링'
    ]
    if slot in ['반지1', '반지2', '반지3', '반지4']:
        if any(seed in name for seed in seed_keywords):
            return '시드링'
        elif any(ev in name for ev in event_keywords):
            return '이벤트링'
        return '기타'

    # 포켓 아이템
    if slot == '포켓 아이템':
        if '핑크빛 성배' in name:
            return '보스 장신구'
        elif any(x in name for x in ['저주받은 적의 마도서', '저주받은 청의 마도서', '저주받은 녹의 마도서', '저주받은 황의 마도서']):
            return '칠흑'
        return '기타'

    # 뱃지
    if slot == '뱃지':
        if '크리스탈 웬투스 뱃지' in name:
            return '보스 장신구'
        elif '창세의 뱃지' in name:
            return '칠흑'
        return '기타'

    # 엠블렘
    if slot == '엠블렘':
        if '미트라의 분노' in name:
            return '칠흑'
        return '기타'

    # 얼굴장식
    if slot == '얼굴장식':
        if '응축된 힘의 결정석' in name:
            return '보스 장신구'
        elif '트와일라이트 마크' in name:
            return '여명'
        elif '루즈 컨트롤 머신 마크' in name:
            return '칠흑'
        return '기타'

    # 눈장식
    if slot == '눈장식':
        if any(x in name for x in ['블랙빈 마크', '파풀라투스 마크', '아쿠아틱 레터']):
            return '보스 장신구'
        elif '마력이 깃든 안대' in name:
            return '칠흑'
        return '기타'

    # 귀고리
    if slot == '귀고리':
        if any(x in name for x in ['데아 시두스 이어링', '지옥의 불꽃']):
            return '보스 장신구'
        elif '에스텔라 이어링' in name:
            return '여명'
        elif '커맨더 포스 이어링' in name:
            return '칠흑'
        return '기타'

    # 펜던트
    if slot in ['펜던트', '펜던트2']:
        if any(x in name for x in ['혼테일', '매커네이터 펜던트', '카오스 혼테일', '도미네이터']):
            return '보스 장신구'
        elif '데이브레이크 펜던트' in name:
            return '여명'
        elif '고통의 근원' in name:
            return '칠흑'
        return '기타'

    # 방어용 최종 fallback
    return '기타'

In [ ]:
df['item_group'] = df.apply(get_item_group, axis=1)
print(df.isna().sum())  # 반드시 0 나와야 정상

nickname                               0
subclass                               0
equipment_slot                         0
item_name                              0
boss_damage_total                      0
ignore_monster_armor_total             0
all_stat_total                         0
damage_total                           0
potential_option_grade                 0
additional_potential_option_grade      0
exceptional_upgrade                    0
boss_damage_add                        0
damage_add                             0
all_stat_add                           0
starforce                              0
starforce_scroll_flag                  0
special_ring_level                     0
main_stat_type                         0
bonus_stat_total                       0
mainstat_total                         0
power_total                            0
potential_option_1_grade               0
potential_option_2_grade               0
potential_option_3_grade               0
main_pot_grade_s

In [ ]:
df['item_group'].unique()

array(['에테르넬', '아케인셰이드', '보조무기', '제네시스', '훈장', '칠흑', '시드링', '기타', '여명',
       '앱솔랩스', '보스 장신구', '이벤트링', '도전자', '파프니르', '봉인된 제네시스'], dtype=object)

In [ ]:
# 인코딩 대상 문자형 열 목록
object_columns = [
    'subclass', 'equipment_slot', 'item_name',
    'potential_option_grade', 'additional_potential_option_grade',
    'starforce_scroll_flag', 'main_stat_type',
    'potential_option_1_grade', 'potential_option_2_grade', 'potential_option_3_grade',
    'main_pot_grade_summary',
    'additional_potential_option_1_grade', 'additional_potential_option_2_grade', 'additional_potential_option_3_grade',
    'add_pot_grade_summary', 'item_group', 'potential_status'
]

# 각 컬럼별로 unique 값 수 + 값 리스트 출력
for col in object_columns:
    unique_vals = df[col].dropna().unique()
    print(f"\n📌 [{col}]")
    print(f" - 고유값 개수: {len(unique_vals)}")
    print(f" - 고유값 목록: {unique_vals if len(unique_vals) <= 20 else list(unique_vals[:20]) + ['...']}")


📌 [subclass]
 - 고유값 개수: 46
 - 고유값 목록: ['카데나', '듀얼블레이더', '호영', '칼리', '나이트로드', '나이트워커', '팬텀', '섀도어', '제논', '보우마스터', '신궁', '패스파인더', '윈드브레이커', '와일드헌터', '메르세데스', '카인', '미하일', '소울마스터', '데몬슬레이어', '데몬어벤져', '...']

📌 [equipment_slot]
 - 고유값 개수: 24
 - 고유값 목록: ['모자', '상의', '하의', '신발', '장갑', '망토', '보조무기', '무기', '훈장', '벨트', '어깨장식', '포켓 아이템', '기계 심장', '뱃지', '엠블렘', '얼굴장식', '눈장식', '귀고리', '반지1', '반지2', '...']

📌 [item_name]
 - 고유값 개수: 1173
 - 고유값 목록: ['에테르넬 시프반다나', '에테르넬 시프셔츠', '에테르넬 시프팬츠', '아케인셰이드 시프슈즈', '아케인셰이드 시프글러브', '아케인셰이드 시프케이프', '트랜스미터 type:A', '제네시스 체인', '칠요의 몬스터파커', '몽환의 벨트', '에테르넬 시프숄더', '저주받은 황의 마도서', '컴플리트 언더컨트롤', '창세의 뱃지', '미트라의 분노 : 도적', '루즈 컨트롤 머신 마크', '마력이 깃든 안대', '커맨더 포스 이어링', '리스트레인트 링', '여명의 가디언 엔젤 링', '...']

📌 [potential_option_grade]
 - 고유값 개수: 6
 - 고유값 목록: ['레전드리' 'None' '유니크' '에픽' '레어' 'Not Granted']

📌 [additional_potential_option_grade]
 - 고유값 개수: 6
 - 고유값 목록: ['레전드리' 'None' '에픽' '유니크' '레어' 'Not Granted']

📌 [starforce_scroll_flag]
 - 고유값 개수: 2
 - 고유값 목록: ['미사용' '사용']

📌 [ma

In [ ]:
# 중간 저장

save_path = "/content/drive/MyDrive/item_nomissingvalues.csv" # 저장 경로 수정해주세요
df.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"CSV 저장 완료: {save_path}")

CSV 저장 완료: /content/drive/MyDrive/item_nomissingvalues.csv


In [ ]:
# 나머지 인코딩 필요한 행에 대해서 원핫 인코딩 진행

drop_cols = ['item_name', 'nickname']

# 인코딩 대상만 추리기
object_columns = [
    'subclass', 'equipment_slot', 'potential_option_grade',
    'additional_potential_option_grade', 'starforce_scroll_flag', 'main_stat_type',
    'potential_option_1_grade', 'potential_option_2_grade', 'potential_option_3_grade',
    'main_pot_grade_summary', 'additional_potential_option_1_grade',
    'additional_potential_option_2_grade', 'additional_potential_option_3_grade',
    'add_pot_grade_summary', 'item_group', 'potential_status'
]

used_object_columns = [col for col in object_columns if col not in drop_cols]

# 불필요 열 제거
df = df.drop(columns=drop_cols)

# 원핫 인코딩
df_encoded = pd.get_dummies(df, columns=used_object_columns)

## 수치형 칼럼에 대한 스케일링

In [ ]:
from sklearn.preprocessing import StandardScaler

num_cols = df_encoded.select_dtypes(include='number').columns
scaler = StandardScaler()
df_encoded[num_cols] = scaler.fit_transform(df_encoded[num_cols])